In [96]:
import numpy as np
import pandas as pd
pd.set_option("display.precision", 4)

## 1.Zadanie
### Vstupne udaje

In [97]:
# Section properties
E = 3.28*(10**7) #kPa
A = 0.06 #m2
I = 4.5*(10**(-4)) #m-4

# lengths
a = 3 #m
b = 3 #m
c = 3 #m

# loads
F = 40 #kN
q = 5 #kN/m

In [98]:
class MemberStiffnessMatrix:

    def localMember(len, A, E, I):
        return pd.DataFrame([[E*A/len, 0, 0], 
                            [0, 12*E*I/len**3, 6*E*I/len**2], 
                            [0, 6*E*I/len**2, 4*E*I/len]])

    def globalMember(rotationMatrix, localMemberStiffnessMatrix, transformationMatrix):
        A = rotationMatrix
        k = localMemberStiffnessMatrix
        B = transformationMatrix
        
        # shape of final output 
        # [kaa, kab],
        # [kba, kbb]
        
        kaa = A.T.dot(k).dot(A)
        kba = B.dot(kaa)
        kab = kba.T
        kbb = B.dot(kab)
        
        col1 = pd.concat([kaa, kba])
        col2 = pd.concat([kab,kbb])
        
        return pd.concat([col1, col2], axis=1)



In [99]:
class TransformationMatrices:
    def rotation(alfa):
        alfa_rad = np.deg2rad(alfa)
        return pd.DataFrame([[np.cos(alfa_rad), np.sin(alfa_rad), 0], 
                            [-np.sin(alfa_rad), np.cos(alfa_rad), 0], 
                            [0, 0, 1]])

    def transmission(lx, ly):
        return pd.DataFrame([[-1, 0, 0], 
                            [0, -1, 0], 
                            [-ly, lx, -1]])

In [100]:
class BendingMoment:
    def _init_(self, length, F, q):
        self.length = length
        self.F = F
        self.q = q
        
    def pointLoad(length, F):
        return F*length/8
    
    def distributedLoad(length, q):
        return q*length**2/12
    
class Reactions:
    def _init_(self, length, F, q):
        self.length = length
        self.F = F
        self.q = q
        
    def pointLoad(F):
        return F/2
    
    def distributedLoad(length, q):
        return q*length/2
    
class LoadMatrix:
    def localMatrix(N,V,M):
        return pd.DataFrame([[N], 
                             [V], 
                             [M]])

    def globalMatrix(localMatrix, A0_transp):
        return A0_transp.dot(localMatrix)

    def superMatrix(globalMatrix_1, globalMatrix_2):
        return pd.concat([globalMatrix_1, globalMatrix_2])

In [101]:
%%HTML
<img src="./assets/schema.png" width="400" height="400" style="position: relative"/>

### Prut 1-2

In [102]:
k01 = MemberStiffnessMatrix.localMember(c,A,E,I)
k01

,0,1,2
0,656000.0,0.0,0.0
1,0.0,6560.0,9840.0
2,0.0,9840.0,19680.0


In [103]:
%%HTML
<img src="./assets/rotation-matrix.png" width="400" height="300"/>

In [104]:
alfa = 270 #deg
A0 = TransformationMatrices.rotation(alfa)
A0

,0,1,2
0,-1.8370e-16,-1.0000e+00,0
1,1.0000e+00,-1.8370e-16,0
2,0.0000e+00,0.0000e+00,1


In [105]:
lx = 0 #m
ly = -3 #m

B = TransformationMatrices.transmission (lx,ly)
B

,0,1,2
0,-1,0,0
1,0,-1,0
2,3,0,-1


In [106]:
k1 = MemberStiffnessMatrix.globalMember(A0, k01, B)
k1

,0,1,2,0,1,2
0,6.5600e+03,1.1930e-10,9.8400e+03,-6.5600e+03,-1.1930e-10,9.8400e+03
1,1.1930e-10,6.5600e+05,-1.8076e-12,-1.1930e-10,-6.5600e+05,3.5971e-10
2,9.8400e+03,-1.8076e-12,1.9680e+04,-9.8400e+03,1.8076e-12,9.8400e+03
0,-6.5600e+03,-1.1930e-10,-9.8400e+03,6.5600e+03,1.1930e-10,-9.8400e+03
1,-1.1930e-10,-6.5600e+05,1.8076e-12,1.1930e-10,6.5600e+05,-3.5971e-10
2,9.8400e+03,3.5971e-10,9.8400e+03,-9.8400e+03,-3.5971e-10,1.9680e+04


Local loading matrix

F = (N,V,M)

In [107]:
N = 0 #kN
V = Reactions.distributedLoad(c, q) #kN
M = BendingMoment.distributedLoad(c, q) #kNm

In [108]:
F1_l = LoadMatrix.localMatrix(N,V,-M)
F2_l = LoadMatrix.localMatrix(N,-V,-M)
F2_l

,0
0,0.00
1,-7.50
2,-3.75


Global loading matrix

In [109]:
F1_g = LoadMatrix.globalMatrix(F1_l, A0.T)
F2_g = LoadMatrix.globalMatrix(F2_l, A0.T)

LoadMatrix.superMatrix(F1_g, F2_g)

,0
0,7.5000e+00
1,-1.3777e-15
2,-3.7500e+00
0,-7.5000e+00
1,1.3777e-15
2,-3.7500e+00


### Prut 2-3

In [110]:
k02 = MemberStiffnessMatrix.localMember(a,A,E,I)
A0 = TransformationMatrices.rotation(alfa = 0)
B = TransformationMatrices.transmission (lx = 3,ly = 0)
k2 = MemberStiffnessMatrix.globalMember(A0, k02, B)
k2


,0,1,2,0,1,2
0,656000.0,0.0,0.0,-656000.0,0.0,0.0
1,0.0,6560.0,9840.0,0.0,-6560.0,9840.0
2,0.0,9840.0,19680.0,0.0,-9840.0,9840.0
0,-656000.0,0.0,0.0,656000.0,0.0,0.0
1,0.0,-6560.0,-9840.0,0.0,6560.0,-9840.0
2,0.0,9840.0,9840.0,0.0,-9840.0,19680.0


In [111]:
V = Reactions.pointLoad(F) #kN
M = BendingMoment.pointLoad(a,F) #kNm
M

15.0

In [112]:
N = 0 #kN
V = Reactions.pointLoad(F) #kN
M = BendingMoment.pointLoad(a,F) #kNm

F1_l = LoadMatrix.localMatrix(N,V,M)
F2_l = LoadMatrix.localMatrix(N,-V,M)

F1_g = LoadMatrix.globalMatrix(F1_l, A0.T)
F2_g = LoadMatrix.globalMatrix(F2_l, A0.T)

LoadMatrix.superMatrix(F1_g, F2_g)

,0
0,0.0
1,20.0
2,15.0
0,0.0
1,-20.0
2,15.0


### Prut 3-4

In [113]:
k03 = MemberStiffnessMatrix.localMember(b,A,E,I)
A0 = TransformationMatrices.rotation(alfa = 90)
B = TransformationMatrices.transmission (lx = 0,ly = 3)
k3 = MemberStiffnessMatrix.globalMember(A0, k03, B)
k3

,0,1,2,0,1,2
0,6.5600e+03,3.9767e-11,-9.8400e+03,-6.5600e+03,-3.9767e-11,-9.8400e+03
1,3.9767e-11,6.5600e+05,6.0253e-13,-3.9767e-11,-6.5600e+05,-1.1990e-10
2,-9.8400e+03,6.0253e-13,1.9680e+04,9.8400e+03,-6.0253e-13,9.8400e+03
0,-6.5600e+03,-3.9767e-11,9.8400e+03,6.5600e+03,3.9767e-11,9.8400e+03
1,-3.9767e-11,-6.5600e+05,-6.0253e-13,3.9767e-11,6.5600e+05,1.1990e-10
2,-9.8400e+03,-1.1990e-10,9.8400e+03,9.8400e+03,1.1990e-10,1.9680e+04


In [114]:
N = 0 #kN
V = 0 #kN
M = 0 #kN

F1_l = LoadMatrix.localMatrix(N,V,M)
F2_l = LoadMatrix.localMatrix(N,-V,M)

F1_g = LoadMatrix.globalMatrix(F1_l, A0.T)
F2_g = LoadMatrix.globalMatrix(F2_l, A0.T)

LoadMatrix.superMatrix(F1_g, F2_g)

,0
0,0.0
1,0.0
2,0.0
0,0.0
1,0.0
2,0.0


In [115]:
zeros = pd.DataFrame(np.zeros((12, 12)))
zeros

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
k1.columns=[0,1,2,3,4,5]
k1 = k1.reset_index(drop=True)
k1

,0,1,2,3,4,5
0,6.5600e+03,1.1930e-10,9.8400e+03,-6.5600e+03,-1.1930e-10,9.8400e+03
1,1.1930e-10,6.5600e+05,-1.8076e-12,-1.1930e-10,-6.5600e+05,3.5971e-10
2,9.8400e+03,-1.8076e-12,1.9680e+04,-9.8400e+03,1.8076e-12,9.8400e+03
3,-6.5600e+03,-1.1930e-10,-9.8400e+03,6.5600e+03,1.1930e-10,-9.8400e+03
4,-1.1930e-10,-6.5600e+05,1.8076e-12,1.1930e-10,6.5600e+05,-3.5971e-10
5,9.8400e+03,3.5971e-10,9.8400e+03,-9.8400e+03,-3.5971e-10,1.9680e+04


In [117]:
layer1 = zeros.add(k1,fill_value=0)
layer1

,0,1,2,3,4,5,6,7,8,9,10,11
0,6.5600e+03,1.1930e-10,9.8400e+03,-6.5600e+03,-1.1930e-10,9.8400e+03,0.0,0.0,0.0,0.0,0.0,0.0
1,1.1930e-10,6.5600e+05,-1.8076e-12,-1.1930e-10,-6.5600e+05,3.5971e-10,0.0,0.0,0.0,0.0,0.0,0.0
2,9.8400e+03,-1.8076e-12,1.9680e+04,-9.8400e+03,1.8076e-12,9.8400e+03,0.0,0.0,0.0,0.0,0.0,0.0
3,-6.5600e+03,-1.1930e-10,-9.8400e+03,6.5600e+03,1.1930e-10,-9.8400e+03,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.1930e-10,-6.5600e+05,1.8076e-12,1.1930e-10,6.5600e+05,-3.5971e-10,0.0,0.0,0.0,0.0,0.0,0.0
5,9.8400e+03,3.5971e-10,9.8400e+03,-9.8400e+03,-3.5971e-10,1.9680e+04,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
k2.columns=[3,4,5,6,7,8]
k2 = k2.reset_index(drop=True).rename(index={0: 3, 1: 4, 2: 5, 3: 6, 4: 7, 5: 8})
k2

,3,4,5,6,7,8
3,656000.0,0.0,0.0,-656000.0,0.0,0.0
4,0.0,6560.0,9840.0,0.0,-6560.0,9840.0
5,0.0,9840.0,19680.0,0.0,-9840.0,9840.0
6,-656000.0,0.0,0.0,656000.0,0.0,0.0
7,0.0,-6560.0,-9840.0,0.0,6560.0,-9840.0
8,0.0,9840.0,9840.0,0.0,-9840.0,19680.0


In [119]:
layer2 = layer1.add(k2,fill_value=0)
layer2

,0,1,2,3,4,5,6,7,8,9,10,11
0,6.5600e+03,1.1930e-10,9.8400e+03,-6.5600e+03,-1.1930e-10,9.8400e+03,0.0,0.0,0.0,0.0,0.0,0.0
1,1.1930e-10,6.5600e+05,-1.8076e-12,-1.1930e-10,-6.5600e+05,3.5971e-10,0.0,0.0,0.0,0.0,0.0,0.0
2,9.8400e+03,-1.8076e-12,1.9680e+04,-9.8400e+03,1.8076e-12,9.8400e+03,0.0,0.0,0.0,0.0,0.0,0.0
3,-6.5600e+03,-1.1930e-10,-9.8400e+03,6.6256e+05,1.1930e-10,-9.8400e+03,-656000.0,0.0,0.0,0.0,0.0,0.0
4,-1.1930e-10,-6.5600e+05,1.8076e-12,1.1930e-10,6.6256e+05,9.8400e+03,0.0,-6560.0,9840.0,0.0,0.0,0.0
5,9.8400e+03,3.5971e-10,9.8400e+03,-9.8400e+03,9.8400e+03,3.9360e+04,0.0,-9840.0,9840.0,0.0,0.0,0.0
6,0.0000e+00,0.0000e+00,0.0000e+00,-6.5600e+05,0.0000e+00,0.0000e+00,656000.0,0.0,0.0,0.0,0.0,0.0
7,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,-6.5600e+03,-9.8400e+03,0.0,6560.0,-9840.0,0.0,0.0,0.0
8,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,9.8400e+03,9.8400e+03,0.0,-9840.0,19680.0,0.0,0.0,0.0
9,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0,0.0,0.0,0.0,0.0,0.0


In [120]:
k3.columns=[6,7,8,9,10,11]
k3 = k3.reset_index(drop=True).rename(index={0: 6, 1: 7, 2: 8, 3: 9, 4: 10, 5: 11})
k3

,6,7,8,9,10,11
6,6.5600e+03,3.9767e-11,-9.8400e+03,-6.5600e+03,-3.9767e-11,-9.8400e+03
7,3.9767e-11,6.5600e+05,6.0253e-13,-3.9767e-11,-6.5600e+05,-1.1990e-10
8,-9.8400e+03,6.0253e-13,1.9680e+04,9.8400e+03,-6.0253e-13,9.8400e+03
9,-6.5600e+03,-3.9767e-11,9.8400e+03,6.5600e+03,3.9767e-11,9.8400e+03
10,-3.9767e-11,-6.5600e+05,-6.0253e-13,3.9767e-11,6.5600e+05,1.1990e-10
11,-9.8400e+03,-1.1990e-10,9.8400e+03,9.8400e+03,1.1990e-10,1.9680e+04


In [121]:
layer3 = layer2.add(k3,fill_value=0)
layer3

,0,1,2,3,4,5,6,7,8,9,10,11
0,6.5600e+03,1.1930e-10,9.8400e+03,-6.5600e+03,-1.1930e-10,9.8400e+03,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00
1,1.1930e-10,6.5600e+05,-1.8076e-12,-1.1930e-10,-6.5600e+05,3.5971e-10,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00
2,9.8400e+03,-1.8076e-12,1.9680e+04,-9.8400e+03,1.8076e-12,9.8400e+03,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00
3,-6.5600e+03,-1.1930e-10,-9.8400e+03,6.6256e+05,1.1930e-10,-9.8400e+03,-6.5600e+05,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00
4,-1.1930e-10,-6.5600e+05,1.8076e-12,1.1930e-10,6.6256e+05,9.8400e+03,0.0000e+00,-6.5600e+03,9.8400e+03,0.0000e+00,0.0000e+00,0.0000e+00
5,9.8400e+03,3.5971e-10,9.8400e+03,-9.8400e+03,9.8400e+03,3.9360e+04,0.0000e+00,-9.8400e+03,9.8400e+03,0.0000e+00,0.0000e+00,0.0000e+00
6,0.0000e+00,0.0000e+00,0.0000e+00,-6.5600e+05,0.0000e+00,0.0000e+00,6.6256e+05,3.9767e-11,-9.8400e+03,-6.5600e+03,-3.9767e-11,-9.8400e+03
7,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,-6.5600e+03,-9.8400e+03,3.9767e-11,6.6256e+05,-9.8400e+03,-3.9767e-11,-6.5600e+05,-1.1990e-10
8,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,9.8400e+03,9.8400e+03,-9.8400e+03,-9.8400e+03,3.9360e+04,9.8400e+03,-6.0253e-13,9.8400e+03
9,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,0.0000e+00,-6.5600e+03,-3.9767e-11,9.8400e+03,6.5600e+03,3.9767e-11,9.8400e+03


In [122]:
def highlightNotZero(x):
    if x < 0:
        #red
        color = "#ff9999"
    elif x > 0:
        #green
        color = "#ccffcc"
    else:
        color = "#f2f2f2"
    return f"background: {color}"

def parseZeroValues(x):
    if x != 0:
        return '{:.3e}'
    else:
        return '{:.0f}'


### Final stiffness matrix

In [123]:
layer3.style\
    .format('{:.3e}')\
    .applymap(highlightNotZero)\
    .set_table_styles([{"selector" : "", "props" : [("font-family", "arial")]}])


,0,1,2,3,4,5,6,7,8,9,10,11
0,6.560e+03,1.193e-10,9.840e+03,-6.560e+03,-1.193e-10,9.840e+03,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00
1,1.193e-10,6.560e+05,-1.808e-12,-1.193e-10,-6.560e+05,3.597e-10,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00
2,9.840e+03,-1.808e-12,1.968e+04,-9.840e+03,1.808e-12,9.840e+03,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00
3,-6.560e+03,-1.193e-10,-9.840e+03,6.626e+05,1.193e-10,-9.840e+03,-6.560e+05,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00
4,-1.193e-10,-6.560e+05,1.808e-12,1.193e-10,6.626e+05,9.840e+03,0.000e+00,-6.560e+03,9.840e+03,0.000e+00,0.000e+00,0.000e+00
5,9.840e+03,3.597e-10,9.840e+03,-9.840e+03,9.840e+03,3.936e+04,0.000e+00,-9.840e+03,9.840e+03,0.000e+00,0.000e+00,0.000e+00
6,0.000e+00,0.000e+00,0.000e+00,-6.560e+05,0.000e+00,0.000e+00,6.626e+05,3.977e-11,-9.840e+03,-6.560e+03,-3.977e-11,-9.840e+03
7,0.000e+00,0.000e+00,0.000e+00,0.000e+00,-6.560e+03,-9.840e+03,3.977e-11,6.626e+05,-9.840e+03,-3.977e-11,-6.560e+05,-1.199e-10
8,0.000e+00,0.000e+00,0.000e+00,0.000e+00,9.840e+03,9.840e+03,-9.840e+03,-9.840e+03,3.936e+04,9.840e+03,-6.025e-13,9.840e+03
9,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,-6.560e+03,-3.977e-11,9.840e+03,6.560e+03,3.977e-11,9.840e+03
